In [1]:
import argparse
from abc import ABC
from datetime import datetime

from tensorflow import keras
import tensorflow as tf

from tf_quantization.layers.quant_conv2D_batch_layer import QuantConv2DBatchLayer
from tf_quantization.layers.quant_depthwise_conv2d_bn_layer import QuantDepthwiseConv2DBatchNormalizationLayer
from tf_quantization.quantize_model import quantize_model
import imagenet_mini
import os
import numpy as np

In [2]:
tf.random.set_seed(30082000)  # Set random seed to have reproducible results

In [3]:
base_model = tf.keras.applications.MobileNet(weights=None, input_shape=(32, 32, 3), alpha=1, classes=10, include_top=False)

Metal device set to: Apple M2 Pro


2023-03-11 16:25:45.954690: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-11 16:25:45.954718: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model = keras.Sequential()
#Creating base layer of VGG19
model.add(base_model)
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())

#Adding the Dense Layers and Dropout
model.add(keras.layers.Dense(512,activation=('relu')))

model.add(keras.layers.Dense(256,activation=('relu')))

model.add(keras.layers.Dropout(.3))

model.add(keras.layers.Dense(128,activation=('relu')))

model.add(keras.layers.Dropout(.2))

model.add(keras.layers.Dense(10,activation=('softmax')))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_32 (Function  (None, 1, 1, 1024)       3228864   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 1, 1, 1024)        0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [6]:
import cifar10

tr_ds = cifar10.get_imagenet_mini_dataset("train")
tr_ds = tr_ds.map(cifar10.get_preprocess_image_fn(image_size=(32, 32)))

train_ds = tr_ds\
    .map(lambda data: (data['image'], data['label']))\
    .batch(128)

ds = cifar10.get_imagenet_mini_dataset("test")
ds = ds.map(cifar10.get_preprocess_image_fn(image_size=(32, 32)))
test_ds = ds.map(lambda data: (data['image'], data['label'])).batch(128)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01),
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                          metrics=['accuracy'])

In [8]:
model.fit(train_ds, epochs=25, validation_data=test_ds)

Epoch 1/25


2023-03-11 16:25:54.429665: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-11 16:25:55.480971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 2.0262 - accuracy: 0.1961

2023-03-11 16:26:09.177207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 16s 34ms/step - loss: 2.0262 - accuracy: 0.1961 - val_loss: 2.3118 - val_accuracy: 0.1530
Epoch 2/25
391/391 [==============================] - 12s 31ms/step - loss: 1.8059 - accuracy: 0.2660 - val_loss: 2.0747 - val_accuracy: 0.2373
Epoch 3/25
391/391 [==============================] - 12s 31ms/step - loss: 1.7053 - accuracy: 0.3399 - val_loss: 2.0322 - val_accuracy: 0.3314
Epoch 4/25
391/391 [==============================] - 12s 31ms/step - loss: 1.8663 - accuracy: 0.2962 - val_loss: 3.0380 - val_accuracy: 0.1415
Epoch 5/25
391/391 [==============================] - 12s 31ms/step - loss: 2.0555 - accuracy: 0.1936 - val_loss: 2.2265 - val_accuracy: 0.2060
Epoch 6/25
391/391 [==============================] - 13s 33ms/step - loss: 1.9478 - accuracy: 0.2259 - val_loss: 1.9436 - val_accuracy: 0.2377
Epoch 7/25
391/391 [==============================] - 13s 33ms/step - loss: 1.9572 - accuracy: 0.2463 - val_loss: 1.9815 - val_accuracy: 0.2057
Epo

In [8]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                          metrics=['accuracy'])

In [ ]:
model.fit(train_ds, epochs=25, validation_data=test_ds)

Epoch 1/25


2023-03-11 16:13:29.188944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 0.1528 - accuracy: 0.9489

2023-03-11 16:13:42.080212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 15s 33ms/step - loss: 0.1528 - accuracy: 0.9489 - val_loss: 0.9086 - val_accuracy: 0.7776
Epoch 2/25
391/391 [==============================] - 11s 29ms/step - loss: 0.0653 - accuracy: 0.9809 - val_loss: 1.0840 - val_accuracy: 0.7748
Epoch 3/25
391/391 [==============================] - 11s 29ms/step - loss: 0.0269 - accuracy: 0.9936 - val_loss: 1.3216 - val_accuracy: 0.7691
Epoch 4/25
391/391 [==============================] - 11s 29ms/step - loss: 0.0124 - accuracy: 0.9974 - val_loss: 1.5271 - val_accuracy: 0.7696
Epoch 5/25
391/391 [==============================] - 11s 29ms/step - loss: 0.0087 - accuracy: 0.9982 - val_loss: 1.6432 - val_accuracy: 0.7654
Epoch 6/25
391/391 [==============================] - 12s 29ms/step - loss: 0.0068 - accuracy: 0.9987 - val_loss: 1.7866 - val_accuracy: 0.7633
Epoch 7/25
391/391 [==============================] - 12s 30ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 1.8757 - val_accuracy: 0.7680
Epo

In [ ]:
model.evaluate(test_ds)